In [4]:
## Import the pandas library to work with dataframes
import pandas as pd

In [5]:
# Set up out variables

year = 2022
url = "https://www.hockey-reference.com/leagues/NHL_2022_skaters.html"

# Read in the dataframe from hockey-reference
df = pd.read_html(url,header=1)[0]

#Add a column that indicates the year. This will be most useful if/when we start reading in
#additional years of data.
df['year'] = year

# Reading in the table adds a row every 50 players or so that is just a copy of the column headers. 
# Remove that by choosing an input that wouldn't ever show up for a player in that column.
df = df[~df['Age'].str.contains("Age")]

# check the data types
df.dtypes

# Reading in the dataframe from a website makes everything an object. Let's put in the work to change all of these to their 
# Proper data type

Rk        object
Player    object
Age       object
Tm        object
Pos       object
GP        object
G         object
A         object
PTS       object
+/-       object
PIM       object
PS        object
EV        object
PP        object
SH        object
GW        object
EV.1      object
PP.1      object
SH.1      object
S         object
S%        object
TOI       object
ATOI      object
BLK       object
HIT       object
FOW       object
FOL       object
FO%       object
year       int64
dtype: object

In [11]:
# Fill in the blank cells with zero. These are cells such as faceoff percentage being blank when that player didn't 
# take a faceoff during the season. Let's just make these '0'
df = df.fillna(0)

# Changing the data types one at a time. There must be a better way to do this...
df = df.astype({"Age": int, "GP": int, "G": int, "A": int, "PTS": int, "+/-": int, "PIM": int, "PS": float, "EV": int, "PP": int, "SH": int, "GW": int, "EV.1": int, "PP.1": int, "SH.1": int, "S": int, "S%": float, "TOI": float, "BLK": int, "HIT": int, "FOW": int, "FOL": int, "FO%": float})

# Average time on ice column throws an error when trying to turn it into a float, so let's just recalculate now. We 
# have to do this later anyway, so let's just avoid the problme and do this now.
df['ATOI'] = round(df['TOI']/df['GP'], 2)

# Drop the 'Rk' axis, it's redundant.
df = df.drop('Rk', axis = 1)

# Let's check to see if we've done everything properly.
df.head()

,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
741,Connor McDavid,25,EDM,C,80,44,79,123,28,45,...,314,14.0,1765.0,22.06,26,75,562,484,53.7,2022
378,Johnny Gaudreau,28,CGY,LW,82,40,75,115,64,26,...,262,15.3,1523.0,18.57,9,10,1,0,100.0,2022
490,Jonathan Huberdeau,28,FLA,LW,80,30,85,115,35,54,...,222,13.5,1554.0,19.42,28,99,18,36,33.3,2022
284,Leon Draisaitl,26,EDM,C,80,55,55,110,17,40,...,278,19.8,1788.0,22.35,25,53,887,774,53.4,2022
555,Kirill Kaprizov,24,MIN,LW,81,47,61,108,27,34,...,289,16.3,1548.0,19.11,29,73,5,4,55.6,2022


In [12]:
# I want t break up the dataset, now that it's clean with everything I want, into Forwards and defensemen. 
# This is looking towards the future, as I want to eventually break these players up into player types, 
# and forwards and defensemen will naturally have different types to cluster into.
def_df = df[df['Pos']== 'D']
forward_df = df[df['Pos'] != 'D']

# We did that correctly, right?
forward_df.head(20)

,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
741,Connor McDavid,25,EDM,C,80,44,79,123,28,45,...,314,14.0,1765.0,22.06,26,75,562,484,53.7,2022
378,Johnny Gaudreau,28,CGY,LW,82,40,75,115,64,26,...,262,15.3,1523.0,18.57,9,10,1,0,100.0,2022
490,Jonathan Huberdeau,28,FLA,LW,80,30,85,115,35,54,...,222,13.5,1554.0,19.42,28,99,18,36,33.3,2022
284,Leon Draisaitl,26,EDM,C,80,55,55,110,17,40,...,278,19.8,1788.0,22.35,25,53,887,774,53.4,2022
555,Kirill Kaprizov,24,MIN,LW,81,47,61,108,27,34,...,289,16.3,1548.0,19.11,29,73,5,4,55.6,2022
727,Auston Matthews,24,TOR,C,73,60,46,106,20,18,...,348,17.2,1504.0,20.60,62,67,691,538,56.2,2022
1057,Steven Stamkos,31,TBL,C,81,42,64,106,24,36,...,241,17.4,1497.0,18.48,31,93,545,523,51.0,2022
1125,Matthew Tkachuk,24,CGY,LW,82,42,62,104,57,68,...,253,16.6,1468.0,17.90,20,93,20,23,46.5,2022
771,J.T. Miller,28,VAN,C,80,32,67,99,15,47,...,206,15.5,1686.0,21.08,56,172,667,566,54.1,2022
719,Mitch Marner,24,TOR,RW,72,35,62,97,23,16,...,224,15.6,1504.0,20.89,31,75,10,8,55.6,2022


In [13]:
# Let's sort by most points, instead of alphabetically. The most impactful players should find themselves near the top, right?
df = df.sort_values('PTS', ascending = False)
df.head(20)

,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
741,Connor McDavid,25,EDM,C,80,44,79,123,28,45,...,314,14.0,1765.0,22.06,26,75,562,484,53.7,2022
490,Jonathan Huberdeau,28,FLA,LW,80,30,85,115,35,54,...,222,13.5,1554.0,19.42,28,99,18,36,33.3,2022
378,Johnny Gaudreau,28,CGY,LW,82,40,75,115,64,26,...,262,15.3,1523.0,18.57,9,10,1,0,100.0,2022
284,Leon Draisaitl,26,EDM,C,80,55,55,110,17,40,...,278,19.8,1788.0,22.35,25,53,887,774,53.4,2022
555,Kirill Kaprizov,24,MIN,LW,81,47,61,108,27,34,...,289,16.3,1548.0,19.11,29,73,5,4,55.6,2022
727,Auston Matthews,24,TOR,C,73,60,46,106,20,18,...,348,17.2,1504.0,20.60,62,67,691,538,56.2,2022
1057,Steven Stamkos,31,TBL,C,81,42,64,106,24,36,...,241,17.4,1497.0,18.48,31,93,545,523,51.0,2022
1125,Matthew Tkachuk,24,CGY,LW,82,42,62,104,57,68,...,253,16.6,1468.0,17.90,20,93,20,23,46.5,2022
771,J.T. Miller,28,VAN,C,80,32,67,99,15,47,...,206,15.5,1686.0,21.08,56,172,667,566,54.1,2022
719,Mitch Marner,24,TOR,RW,72,35,62,97,23,16,...,224,15.6,1504.0,20.89,31,75,10,8,55.6,2022


In [20]:
# Now that the dataframe is how I want it, let's save it to a CSV so I can transfer it into Power BI.
df.to_csv("Desktop/NHL csv/NHL_Players.csv", index = False)

In [21]:
def_df.to_csv("Desktop/NHL csv/NHL_Defenders.csv", index = False)
forward_df.to_csv("Desktop/NHL csv/NHL_Forwards.csv", index = False)